# 실전 문제 해결을 위한 모범 사례
- Hyperparameter tuning
- Model ensemble
- Multi GPU/TPU training
- Mixed-precision training
- Cloud computing..

## 모델의 최대 성능을 끌어 내기

### 하이퍼파라미터 최적화
- 층 갯수
- 유닛 사이즈
- 필터 갯수
- 활성화 함수 종류
- BatchNormalization 위치
- 드롭아웃  
등..  

  
**하이퍼파라미터 자동 최적화**  
- 가능한 결정 공간을 자동적, 조직적, 규칙적인 방법으로 탐색
- 하이퍼파라미터 선택 알고리즘
  - Bayesian optimization
  - Genetic algorithms
  - Random search
  - etc...
- 하이퍼파라미터 공간은 discrete, non differentiable -> gradient descent 대신 gradient-free optimization 기법 사용
- gradient-free optimization의 피드백 신호를 계산하려면 새로운 모델을 만들고 데이터셋을 처음부터 다시 훈련해야 해
- 근데 이게 진짜 튜닝 때문일까? 우연히 초기 가중치가 좋아서 그런걸까? -> 잡음이 많음


#### KerasTuner 사용하기

In [2]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


**KerasTuner 모델 구축 함수**

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

# hp 객체를 받는 model_builder() 메소드 정의
def build_model(hp):
    # hp 객체에서 하이퍼파라미터 값을 샘플링
    # 변수 units은 일반적인 파이썬 상수
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 16-64
    # 첫 번째 Dense 레이어의 unit 개수를 16-64 사이에서 16 단위로 조정할 수 있도록 설정
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])

    # Tune the optimizer
    # Choose an optimizer between rmsprop, adam
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

- `hp.Int`, `hp.Choice` 외에 `hp.Float`, `hp.Boolean`, `hp.Fixed`도 사용 가능  
- https://keras.io/keras_tuner/api/hyperparameters/

**KerasTuner의 `HyperModel`**

In [4]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        # 모델 상수 정의
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

<ipython-input-4-643dcc05e516>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### Tuner 정의하기
아래 과정을 반복하는 for loop
1. 일련의 하이퍼파라미터 값을 선택
2. 이런 값으로 모델 구축 함수를 호출하여 모델을 빌드
3. 모델을 훈련하고 평가 결과를 기록

### KerasTuner의 내장 튜너
- RandomSearch
- BayesianOptimization
- Hyperband

In [25]:
tuner = kt.BayesianOptimization(
    # 모델 빌드 함수
    build_model,
    # 튜너가 최적화 할 지표
    # 일반화할 수 있는 모델을 찾아야 하니 항상 검증 지표 사용
    objective="val_accuracy",
    # 탐색을 끝내기 전까지 시도할 모델 설정의 최대 횟수
    # 코랩에서 정상 실행만 확인하기 위해 최대 탐색 횟수를 100에서 10으로 줄입니다
    max_trials=10, # 100
    # 측정값의 분산을 줄이기 위해 동일한 모델을 여러 번 훈련하여 결과를 평균
    # executions_per_trial : 각 모델을 몇 번 훈련하여 평균낼 지
    executions_per_trial=2,
    directory="mnist_kt_test",
    # 새로운 탐색을 시작하기 위해 디렉터리에 있는 데이터를 덮어쓸지 여부
    # 모델 빌드 함수 수정 : True
    # 동일한 모델 빌드
    overwrite=True,
)

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 10 Complete [00h 00m 53s]
val_accuracy: 0.9743500053882599

Best val_accuracy So Far: 0.9759000241756439
Total elapsed time: 00h 11m 13s


**최상의 하이퍼파라미터 설정 확인하기**

In [8]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [9]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [10]:
best_hps

In [11]:
best_hps[0].Choice(name="optimizer", values=["rmsprop", "adam"])

'adam'

In [12]:
model = build_model(best_hps[0])

In [13]:
from keras_tuner.engine.hyperparameters import Int

In [14]:
best_hps[0]._conditions_are_active(Int(name="units", min_value=0, max_value=1).conditions)

True

In [15]:
best_hps[0]._hps

defaultdict(list,
            {'units': [Int(name: 'units', min_value: 16, max_value: 64, step: 16, sampling: linear, default: 16)],
             'optimizer': [Choice(name: 'optimizer', values: ['rmsprop', 'adam'], ordered: False, default: rmsprop)]})

In [16]:
best_hps[0]._exists('units', [])

True

In [17]:
best_hps[0].values

{'units': 64, 'optimizer': 'adam'}

In [18]:
best_hps[0].Int(name="units", min_value=0, max_value=1)

64

In [19]:
model.get_config()

{'name': 'sequential_1',
 'trainable': True,
 'dtype': {'module': 'keras',
  'class_name': 'DTypePolicy',
  'config': {'name': 'float32'},
  'registered_name': None},
 'layers': [{'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'dtype': {'module': 'keras',
     'class_name': 'DTypePolicy',
     'config': {'name': 'float32'},
     'registered_name': None},
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {

In [20]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7936 - loss: 0.7489 - val_accuracy: 0.9358 - val_loss: 0.2363
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9315 - loss: 0.2396 - val_accuracy: 0.9503 - val_loss: 0.1822
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9485 - loss: 0.1804 - val_accuracy: 0.9573 - val_loss: 0.1545
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9579 - loss: 0.1462 - val_accuracy: 0.9631 - val_loss: 0.1354
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9662 - loss: 0.1182 - val_accuracy: 0.9650 - val_loss: 0.1216
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9705 - loss: 0.0999 - val_accuracy: 0.9675 - val_loss: 0.1120
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9747 - loss: 0.0911 - val_accuracy: 0.9682 - val_loss: 0.1106
Epoch 8/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9768 - loss: 0.0778 - val_accu

In [21]:
best_models = tuner.get_best_models(top_n)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#### 올바른 검색 공간을 만드는 기술

#### 하이퍼파라미터 튜닝의 미래: 자동화된 머신 러닝

### 모델 앙상블
- 앙상블에서는 모델의 다양성 역시 중요
- 실전에서 가장 잘 동작하는 한 가지 방법은 트리 기반 모델(Random foreset/Gradient boosting tree)이나 Deep Neural Network를 앙상블하는 것

![0205-voting-bagging.png](./images/0205-voting-bagging.png)
![0205-bagging-boosting.png](./images/0205-bagging-boosting.png)
- Voting
  - Hard voting : 다수결의 원칙을 사용하여 가장 많은 결과가 나온 클래스를 선택
  - Soft voting : 각 클래스 별 확률을 결정하고 평균하여 
  - 케창딥에 나온 예제
  - 여러 모델들을 사용, 결과에 대해 다수결에 의거, 모델의 예측값을 종합하여 수행  (분류라면 다수결, 회귀라면 평균)
  - 성능이 안좋은 모델이 포함될 경우, 전체적으로 성능 저하
    - 성능이 좋은 모델의 결과에 가중치를 주어 진행함으로 개선가능
- Bagging (Bootstrap Aggregating)
  - 중복을 허용하여 데이터를 랜덤 샘플링하는 [부트스트랩](https://modulabs.co.kr/blog/bootstrap-relation-strap-on-boots/) 방식으로 여러개의 하위 데이터셋을 구축 후 한 종류의 여러 모델을 훈련, 이를 통합하여 최종 모델 제작
    - 부트스트래핑 분할 : 여러 개의 데이터 세트를 중첩되게 분리
  - 학습 데이터가 충분하지 않을 때 성능 향상을 할 수 있는 기법
  - 데이터가 중복되어 사용되기 때문에, 개별 모델들이 서로 다른 방향으로 편향되는 현상을 줄일 수 있음
  - 배깅의 대표적인 알고리즘 : 랜덤포레스트
- Boosting
  - sequential한 weak learner들을 여러 개 결합하여 예측 혹은 분류 성능을 높이는 알고리즘
  - 부스팅은 Additive model(비모수 회귀, 즉 함수 형태를 가정하지 않는 회귀모형을 의미) 중 하나
  - 이전 모델이 제대로 예측하지 못한 데이터에 가중치를 부여하고, 다음 모델이 학습
  - 오답에 높은 가중치를 부여하므로, 정확도가 높아지나 오버피팅 가능성이 높아짐
- Stacking
  - 데이터셋에 대한 개별 모델의 예측값들을 최종 모델이 재학습
  - 여러 모델들을 활용해 각각의 예측 결과를 도출한 뒤 그 예측 결과를 결합해 최종 예측 결과를 만들어내는 것
  - 이 때, 개별 모델의 예측값들은 메타 데이터, 최종 모델은 메타 모델이라고 칭함
  - 단순히 개별 모델을 조합하는 것이 아니라, 메타 모델이 기초 모델의 약점을 보완하도록 학습

## 대규모 모델 훈련하기

### 혼합 정밀도로 GPU에서 훈련 속도 높이기

#### 부동 소수점 정밀도 이해하기

In [22]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

tf.float64

In [23]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

#### 혼합 정밀도로 훈련하기

In [24]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### 다중 GPU 훈련

#### 두 개 이상의 GPU 활용하기

#### 단일 호스트, 다중 장치 동기 훈련

### TPU 훈련

#### 구글 코랩에서 TPU 사용하기

#### 스텝 융합을 활용하여 TPU 활용도 높이기

## 요약